In [4]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from scipy import linalg

# === DATA ===
sett_types = pd.read_csv("../../data/hun/HU_places_admin_pop_ZIP_latlon.csv",
           sep=',',
           header=0)

sett_types['admin county']=sett_types['admin county'].str.replace('főváros','Budapest')

KSH = pd.read_csv("../../data/hun/KSHCommuting_c1ID_c1name_c2ID_c2name_comm_school_work_DIR.csv",
           sep=',',
           header=0)

hun_infecteds = pd.read_csv("../../data/hun/hun_approx_infecteds.csv",
           sep=',',
           header=0)

korfa_teltip1 = {
    "főváros": [210640, 81626, 246127, 313540, 213367, 225379, 216828, 221533, 1729040],
    "fővárosi kerület": [210640, 81626, 246127, 313540, 213367, 225379, 216828, 221533, 1729040],
    "megyeszékhely-megyei jogú város": [238608 , 116868 , 233585 , 287108 , 227042 , 252073 , 209260 , 197113 , 1761657],
    "megyei jogú város": [37141, 15599, 32610, 44865, 35886, 39168, 32719, 29748, 267736],
    "város": [477932, 195704, 364021, 491615, 422355, 466796, 373198, 353804, 3145425],
    "all_város": [964321, 409797, 876343, 1137128, 898650, 983416, 832005, 802198, 6903858],
    "község": [483338, 183737, 353193, 443785, 417543, 455266, 344957, 351951, 3033770],
    "nagyközség": [483338, 183737, 353193, 443785, 417543, 455266, 344957, 351951, 3033770],
    "all": [1447659 , 593534 , 1229536 , 1580913 , 1316193 , 1438682 , 1176962 , 1154149 , 9937628],
}
ages = ["0–14", "15–19", "20–29", "30–39", "40–49", "50–59", "60–69", "70–", "all"]


In [2]:
# === Populations and infected ===
def get_age(row, korfa_teltip, I, L, Ks):
    arr = np.array(korfa_teltip[row["settlement type"]])
    arr = arr[:-1]/np.sum(arr[:-1])
    N = row["population"]
    
    ages = (np.array(arr)*N).astype(int)
    
    corrected_ages = np.array([a if i in Ks else 0 for i,a in enumerate(ages)])
    Is = np.random.multinomial(I, corrected_ages/np.sum(corrected_ages))
    Ls = np.random.multinomial(L, corrected_ages/np.sum(corrected_ages))
    
    ages = [{"N": int(s), "S":int(s-i-l), "L":int(l), "I":int(i), "R":0} for s,l,i in zip(ages,Ls,Is)]
    return ages

def create_population_dict(sett_types,hun_infecteds, population_th, num_I, num_L, Ks, budapest, out, sett_inf=False):
    
    
    
    init_inf=[]
    if sett_inf:
        pass
    
    else:
        for index, row in sett_types.iterrows():
            init_inf.append(hun_infecteds.iloc[170][row['admin county']] * row['population'] /sum(sett_types[sett_types['admin county']==row['admin county']]['population']))
    sett_types['initial infecteds']=init_inf
   
    # === Restrict to small cities ===
    big_cities = sett_types[sett_types["population"]>population_th]
    big_cities = big_cities[big_cities["place"] != "Budapest"]
    
    infs = np.array(list(big_cities["initial infecteds"].array))
    if budapest:
        for i,c in enumerate(big_cities["place"].array):
            if c[:8] != "Budapest":
                infs[i] = 0

    # === Spread infected agents ===
    Is = np.random.multinomial(num_I, infs/np.sum(infs))
    Ls = np.random.multinomial(num_L, infs/np.sum(infs))
    
    place_id_dict = {}
    population = {"populations":[]}
    for ind, (_, row) in enumerate(big_cities.iterrows()):
        city = {
            "name": str(ind),
            #"name": row["place"],
            "city":row["place"],
            "index": ind,
            "N": row["population"],
            "r1":1.0,
            "r2":1.0,
            "age": get_age(row, korfa_teltip1, Is[ind], Ls[ind], Ks)
        }
        place_id_dict[row['place']] = ind
        population["populations"].append(city)
    
    with open(f"../input/{out}/populations_KSH.json", "w") as f:
        f.write(json.dumps(population))
    
    district_pops, district_id_dict = generate_district_pop_dict(population, big_cities, out=f"{out}")
    
    return population, place_id_dict, big_cities, district_pops, district_id_dict

def add_ages(ages1, ages2):
    ages = []

    for a1,a2 in zip(ages1, ages2):
        ages.append({"N":a1["N"]+a2["N"], "S":a1["S"]+a2["S"], "L":a1["L"]+a2["L"], "I":a1["I"]+a2["I"], "R":a1["R"]+a2["R"]})
    return ages

def generate_district_pop_dict(populations, big_cities, out):
    all_district = set(big_cities["admin municip"])
    get_district = big_cities.set_index('place').to_dict()["admin municip"]

    pops = {}
    district_id_dict = {}
    for ind,d in enumerate(all_district):
        district_id_dict[d]=ind
        dist = {
            "name":str(ind),
            "district":d,
            "index":ind,
            "N":0,
            "r1":1.0,
            "r2":1.0,
            "age": None
        }
        pops[d]=dist
    
    for city in populations["populations"]:
        dist_name = get_district[city["city"]]
        pops[dist_name]["N"] += city["N"]
        if pops[dist_name]["age"] == None:
            pops[dist_name]["age"] = city["age"]
        else:
            pops[dist_name]["age"] = add_ages(pops[dist_name]["age"], city["age"])

    with open(f"../input/{out}/district/populations_KSH.json", "w") as f:
        f.write(json.dumps({"populations":list(pops.values())}))
    
    with open(f"../input/{out}/district_eigen/populations_KSH.json", "w") as f:
        f.write(json.dumps({"populations":list(pops.values())}))
    
    return pops, district_id_dict

def get_eigen_mtx(mtx, district_ind, pop, out):
    """
    Description:
        Returns the eig.vals enhanced district aggregated mtx
    Parameters:
        * mtx          : commutation mtx between cities
        * district_ind : district dictionary, containing the cities in the district
        * pop          : population of each city
    """
    A = np.array([mtx[:,j]*pop[j] for j in range(len(mtx))])
    #phi = np.linalg.eigvals(A)
    eigvals, eigenVectors = np.linalg.eig(A)
    phi = np.real(eigenVectors[:,0])
    phi = phi/np.sum(phi)
    print("Largest eigenvalue:", eigvals[0])
    #print(list(phi))
    with open(f"../input/{out}/{th}.npy", "wb")as file:
        np.save(file, np.array(phi))
    
    
    mtx_d = np.zeros((len(district_ind), len(district_ind)))
    for l,k in itertools.product(district_ind.keys(), district_ind.keys()):
        up = 0
        for i,j in itertools.product(district_ind[l], district_ind[k]):
            up +=  mtx[i,j]*phi[j]
            #up +=  mtx[i,j]*phi[j]*pop[i]*pop[j]
        
        down = sum([phi[j]*pop[j] for j in district_ind[j]])
        mk = sum([pop[i] for i in district_ind[k]])
        ml = sum([pop[i] for i in district_ind[l]])
        
        if(down < 1e-12): mtx_d[l,k] = 0.0
        else: mtx_d[l,k] = ml*up/down
    
    return mtx_d

# === Commuting ===
def create_commuting(cities, place_id_dict, big_cities, district_pops, district_id_dict, out):
    N = len(cities)
    M = len(district_id_dict)
    pop_dict = {row['place']:row["population"] for _,row in sett_types.iterrows()}


    get_district = big_cities.set_index('place').to_dict()["admin municip"]

    mtx = np.zeros((N,N))
    mtx_dist = np.zeros((M,M))
    for _,row in KSH.iterrows():
        weight = row["CommutersAll"]
        orig,dest = row["origName"], row["destName"]
        if((orig in cities) and (dest in cities)):
            mtx[place_id_dict[orig], place_id_dict[dest]] = weight/pop_dict[orig]
            a = district_id_dict[get_district[orig]]
            b = district_id_dict[get_district[dest]]
            mtx_dist[a,b] += weight
    
    for d,ind in district_id_dict.items():
        mtx[ind]/= district_pops[d]["N"]
    
    # === CITY COMMUTATION ===
    network = [{"from":i, "to":j, "weight":mtx[i,j]} for i,j in itertools.product(range(N), range(N)) if i!= j]
    commuting = {
        "N":N,
        "network":network
    }

    # === DISTRICT COMMUTATION ===
    print("Districts: ",M)
    network_dist = [{"from":i, "to":j, "weight":mtx_dist[i,j]} for i,j in itertools.product(range(M), range(M)) if i!= j]
    commuting_dist = {
        "N":M,
        "network":network_dist
    }

    # === EIGEN DISTRICT COMMUTATION ===
    district_ind = {i:[] for i in range(len(mtx_dist))}
    pop = np.zeros(len(mtx))
    for city in cities:
        pop[place_id_dict[city]] = pop_dict[city]
        
        act_dist = district_id_dict[get_district[city]]
        district_ind[act_dist].append(act_dist)
    
    city_names = ["" for i in range(N)]
    for city in place_id_dict:
        city_names[place_id_dict[city]] = city
    #print(city_names)
    with open(f"../input/{out}/{th}_cities.npy", "wb")as file:
        np.save(file, np.array(city_names))
    
    mtx_eigen = get_eigen_mtx(mtx, district_ind, pop, out)
    network_eigen = [{"from":i, "to":j, "weight":mtx_eigen[i,j]} for i,j in itertools.product(range(M), range(M)) if i!= j]
    commuting_eigen = {
        "N":M,
        "network":network_eigen
    }

    with open(f"../input/{out}/commuting_KSH.json", "w") as f:
        f.write(json.dumps(commuting))
    
    with open(f"../input/{out}/district/commuting_KSH.json", "w") as f:
        f.write(json.dumps(commuting_dist))
    
    with open(f"../input/{out}/district_eigen/commuting_KSH.json", "w") as f:
        f.write(json.dumps(commuting_eigen))

# === Create contats_home and contacts other ===
import itertools

def select_specified_ages(mtx):
    indexes = [[0,2], [3,3], [4,5], [6,7], [8,9], [10,11], [12,13], [14,15]]
    N = len(indexes)
    new = np.zeros((N,N))

    for i,j in itertools.product(range(N), range(N)):
        a,b = indexes[i]
        c,d = indexes[j]
        new[i,j] = np.mean(mtx[a:b+1,c:d+1])
    return new

def write_mtx(mtx, name, out):
    N = len(mtx)
    d = {
        "K":len(mtx),
        "rates": [{"from":i,"to":j, 'rate':mtx[i,j]} for i,j in itertools.product(range(N), range(N))]
    }
    with open(f"../input/{out}/contacts_{name}.json", "w") as f:
        f.write(json.dumps(d))

def create_new_contact_mtx(out):
    d = json.load(open("../input/santiago/contacts_home.json"))
    K = int(d['K'])
    mtx_home = np.zeros((K,K))
    for d1 in d['rates']:
        mtx_home[d1["from"]][d1["to"]] = d1['rate']
        
    d = json.load(open("../input/santiago/contacts_other.json"))
    K = int(d['K'])
    mtx_other = np.zeros((K,K))
    for d1 in d['rates']:
        mtx_other[d1["from"]][d1["to"]] = d1['rate']
    
    new_home = select_specified_ages(mtx_home)
    new_other = select_specified_ages(mtx_other)
    write_mtx(new_home, "home", out)
    write_mtx(new_other, "other", out)

    write_mtx(new_home, "home", f"{out}/district")
    write_mtx(new_other, "other", f"{out}/district")

    write_mtx(new_home, "home", f"{out}/district_eigen")
    write_mtx(new_other, "other", f"{out}/district_eigen")

def create_config(N, K, out):
    d = {
        "commuting_file": "commuting_KSH.json",
        "populations_file": "populations_KSH.json",
        "contacts_file_home": "contacts_home.json",
        "contacts_file_other": "contacts_other.json",
        "Npop": str(N[0]),
        "K":str(K)
    }
    with open(f"../input/{out}/config.json", "w") as f:
        f.write(json.dumps(d, indent=4))
    
    d["Npop"] = str(N[1])
    with open(f"../input/{out}/district/config.json", "w") as f:
        f.write(json.dumps(d, indent=4))
    with open(f"../input/{out}/district_eigen/config.json", "w") as f:
        f.write(json.dumps(d, indent=4))

In [3]:
th = 10000
dest_folder = f"hun_{th}"
if(not os.path.exists(f"../input/{dest_folder}/district")):
    os.makedirs(f"../input/{dest_folder}/district")
if(not os.path.exists(f"../input/{dest_folder}/district_eigen")):
    os.makedirs(f"../input/{dest_folder}/district_eigen")

np.random.seed(1)
# Creates nodes with SEIR states, with given infection distribution
population, place_id_dict, big_cities, district_pops, district_id_dict = create_population_dict(
    sett_types, hun_infecteds, population_th=th,
    num_I = 10000,
    num_L=int(10000*(4/2.5)),
    Ks = [4,5],
    budapest = False,
    out=dest_folder,
    sett_inf=False)

cities = set(place_id_dict.keys())
print(f"Number of cities: {len(cities)}")
print(f'Number of age groups {len(population["populations"][1]["age"])}')

create_commuting(cities, place_id_dict, big_cities, district_pops, district_id_dict, out=dest_folder)
create_new_contact_mtx(out=dest_folder)
create_config(N=(len(cities), len(district_id_dict)), K=8, out=dest_folder)

Number of cities: 167
Number of age groups 8
Districts:  111
Largest eigenvalue: 80061.58895369637


In [5]:
len(set(sett_types['admin municip']))

175

In [10]:
list(set(sett_types['admin municip']))

['Hódmezővásárhely',
 'Siklós',
 'Csongrád',
 'Lenti',
 'Bélapátfalva',
 'Zirc',
 'Mosonmagyaróvár',
 'Tiszavasvári',
 'Siófok',
 'Tapolca',
 'Fehérgyarmat',
 'Veresegyház',
 'Cegléd',
 'Kapuvár',
 'Karcag',
 'Kisvárda',
 'Pilisvörösvár',
 'Cigánd',
 'Sárospatak',
 'Mohács',
 'Csorna',
 'Oroszlány',
 'Érd',
 'Mezőtúr',
 'Szentendre',
 'Barcs',
 'Csurgó',
 'Békés',
 'Pápa',
 'Tiszaújváros',
 'Jánoshalma',
 'Budapest',
 'Székesfehérvár',
 'Záhony',
 'Tata',
 'Mátészalka',
 'Dunakeszi',
 'Pannonhalma',
 'Fonyód',
 'Gönc',
 'Balmazújváros',
 'Szarvas',
 'Sarkad',
 'Polgár',
 'Devecser',
 'Dunaújváros',
 'Tét',
 'Kadarkút',
 'Szombathely',
 'Szikszó',
 'Nagykáta',
 'Eger',
 'Tab',
 'Ózd',
 'Baktalórántháza',
 'Debrecen',
 'Bicske',
 'Tatabánya',
 'Bonyhád',
 'Gyöngyös',
 'Szentgotthárd',
 'Komló',
 'Kőszeg',
 'Kunszentmárton',
 'Bátonyterenye',
 'Szeghalom',
 'Békéscsaba',
 'Jászberény',
 'Hévíz',
 'Ercsi',
 'Szolnok',
 'Celldömölk',
 'Csepreg',
 'Sellye',
 'Kiskunfélegyháza',
 'Zalakaros',

In [6]:
sett_types.head()

,place,KSH code,settlement type,admin county,admin municip,population,zip,latitude,longitude
0,Aba,17376,nagyközség,Fejér,Aba,4619,8127,47.0291,18.5217
1,Abádszalók,12441,város,Jász-Nagykun-Szolnok,Tiszafüred,3922,5241,47.4667,20.6000
2,Abaliget,12548,község,Baranya,Pécs,586,7678,46.1426,18.1168
3,Abasár,24554,község,Heves,Gyöngyös,2498,3261,47.7970,20.0032
4,Abaújalpár,15662,község,Borsod-Abaúj-Zemplén,Encs,76,3882,48.3067,21.2332


In [35]:
hun_infecteds.iloc[170]

Dátum                     2020-09-17
Bács-Kiskun                    296.0
Baranya                        374.0
Békés                          209.0
Borsod-Abaúj-Zemplén           548.0
Budapest                      3759.0
Csongrád                       662.0
Fejér                          413.0
Győr-Moson-Sopron              806.0
Hajdú-Bihar                    520.0
Heves                          198.0
Jász-Nagykun-Szolnok           407.0
Komárom-Esztergom              329.0
Nógrád                         211.0
Pest                          1458.0
Somogy                         268.0
Szabolcs-Szatmár-Bereg         587.0
Tolna                          110.0
Vas                            371.0
Veszprém                       289.0
Zala                           186.0
Name: 170, dtype: object

In [21]:
sum(sett_types[sett_types['admin county']=='Fejér']['population'])

425581

In [41]:
init_inf=[]

In [38]:
sett_types['admin county']=sett_types['admin county'].str.replace('főváros','Budapest')


In [42]:
for index, row in sett_types.iterrows():
    init_inf.append(hun_infecteds.iloc[170][row['admin county']] * row['population'] /sum(sett_types[sett_types['admin county']==row['admin county']]['population']))

In [43]:
sett_types['initial infecteds']=init_inf